# 01 - Reconstruction

Reconstruct speed, acceleration, direction, and other derived signals from position data.

In [2]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

In [3]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

nfl_big_data_bowl_2026_analytics = kagglehub.competition_download('nfl-big-data-bowl-2026-analytics')

print('Data source import complete.')

100%|██████████| 103M/103M [00:03<00:00, 31.6MB/s]

Extracting files...


Data source import complete.


In [4]:
print(nfl_big_data_bowl_2026_analytics)

/root/.cache/kagglehub/competitions/nfl-big-data-bowl-2026-analytics


In [5]:
!ls /root/.cache/kagglehub/competitions/nfl-big-data-bowl-2026-analytics

114239_nfl_competition_files_published_analytics_final


In [6]:
!ls /root/.cache/kagglehub/competitions/nfl-big-data-bowl-2026-analytics/114239_nfl_competition_files_published_analytics_final

supplementary_data.csv	train


In [7]:
import pandas as pd
import os

# Define the path to the data directory
data_dir = '/root/.cache/kagglehub/competitions/nfl-big-data-bowl-2026-analytics/114239_nfl_competition_files_published_analytics_final/train'

# Construct the full path to the input data for week 1
week1_input_path = os.path.join(data_dir, 'input_2023_w01.csv')

# Load the week 1 input data into a pandas DataFrame
week1_input_df = pd.read_csv(week1_input_path)

# Display the first few rows of the DataFrame
display(week1_input_df.sample(20))

,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,...,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
136766,2023091006,3295,False,54485,12,right,86,Kenny Pickett,6-3,220,...,Passer,80.12,24.09,2.43,1.10,277.62,142.67,10,91.970001,7.860000
63681,2023091002,3505,False,48026,16,left,37,Nick Scott,5-11,200,...,Defensive Coverage,19.39,29.88,3.01,1.01,269.28,95.44,9,36.200001,46.310001
100743,2023091004,2961,False,52430,21,left,78,Justin Jefferson,6-3,192,...,Other Route Runner,70.46,37.15,7.20,0.28,271.44,269.79,10,71.930000,38.139999
190361,2023091009,2826,False,54580,22,right,104,Damarri Mathis,5-11,195,...,Defensive Coverage,108.92,41.25,3.62,4.69,66.80,329.22,8,106.970001,21.230000
262118,2023091013,1737,False,47842,3,right,62,Parris Campbell,6-0,205,...,Other Route Runner,59.64,39.25,0.02,0.67,76.18,82.80,10,75.800003,42.220001
117403,2023091005,3208,True,47822,5,left,29,Sean Murphy-Bunting,6-0,195,...,Defensive Coverage,23.05,39.82,1.54,0.49,296.84,91.09,21,7.610000,49.759998
176533,2023091008,4021,False,52457,2,left,33,Chase Claypool,6-4,227,...,Other Route Runner,34.92,16.77,0.09,0.08,282.18,274.15,8,30.730000,18.920000
222542,2023091011,1443,False,44862,1,right,58,Justin Evans,6-0,199,...,Defensive Coverage,66.87,20.97,1.20,0.40,236.34,260.77,12,63.900002,9.700000
86860,2023091004,315,False,47877,4,left,94,Jamel Dean,6-1,206,...,Defensive Coverage,86.19,9.18,0.05,0.03,275.23,73.60,10,87.309998,39.029999
60580,2023091002,2693,True,53463,3,right,35,Elijah Moore,5-9,184,...,Targeted Receiver,34.47,36.64,0.13,1.01,116.04,84.46,17,62.889999,36.279999


In [8]:
import pandas as pd
import os

# Define the path to the data directory
data_dir = '/root/.cache/kagglehub/competitions/nfl-big-data-bowl-2026-analytics/114239_nfl_competition_files_published_analytics_final'

# Construct the full path to the output data for week 1
week1_output_path = os.path.join(data_dir, 'train', 'output_2023_w01.csv')

# Load the week 1 output data into a pandas DataFrame
week1_output_df = pd.read_csv(week1_output_path)

# Display the first few rows of the DataFrame
print("First 5 rows of output_2023_w01.csv:")
display(week1_output_df.head())

First 5 rows of output_2023_w01.csv:


,game_id,play_id,nfl_id,frame_id,x,y
0,2023090700,101,46137,1,56.22,17.28
1,2023090700,101,46137,2,56.63,16.88
2,2023090700,101,46137,3,57.06,16.46
3,2023090700,101,46137,4,57.48,16.02
4,2023090700,101,46137,5,57.91,15.56


In [9]:
import pandas as pd
import os

# Define the path to the data directory
data_dir = '/root/.cache/kagglehub/competitions/nfl-big-data-bowl-2026-analytics/114239_nfl_competition_files_published_analytics_final'

# Construct the full path to the supplementary data
supplementary_data_path = os.path.join(data_dir, 'supplementary_data.csv')

# Load the supplementary data into a pandas DataFrame
supplementary_df = pd.read_csv(supplementary_data_path)

# Display the first few rows of the DataFrame
print("First 5 rows of supplementary_data.csv:")
display(supplementary_df.head())

First 5 rows of supplementary_data.csv:


/tmp/ipython-input-3401357624.py:11: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  supplementary_df = pd.read_csv(supplementary_data_path)


,game_id,season,week,game_date,game_time_eastern,home_team_abbr,visitor_team_abbr,play_id,play_description,quarter,...,team_coverage_type,penalty_yards,pre_penalty_yards_gained,yards_gained,expected_points,expected_points_added,pre_snap_home_team_win_probability,pre_snap_visitor_team_win_probability,home_team_win_probability_added,visitor_team_win_probility_added
0,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,3461,(10:46) (Shotgun) J.Goff pass deep left to J.R...,4,...,COVER_2_ZONE,NaN,18,18,-0.664416,2.945847,0.834296,0.165704,-0.081149,0.081149
1,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,461,(7:30) J.Goff pass short right to J.Reynolds t...,1,...,COVER_6_ZONE,NaN,21,21,1.926131,1.345633,0.544618,0.455382,-0.029415,0.029415
2,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,1940,(:09) (Shotgun) J.Goff pass incomplete deep ri...,2,...,COVER_2_ZONE,NaN,0,0,0.281891,-0.081964,0.771994,0.228006,0.000791,-0.000791
3,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,1711,"(:45) (No Huddle, Shotgun) P.Mahomes pass deep...",2,...,COVER_2_ZONE,NaN,26,26,3.452352,2.342947,0.663187,0.336813,0.041843,-0.041843
4,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,1588,(1:54) (Shotgun) P.Mahomes pass incomplete dee...,2,...,COVER_4_ZONE,NaN,0,0,1.921525,-0.324035,0.615035,0.384965,0.000061,-0.000061


In [11]:
# Define the base data directory based on previous download
base_data_dir = Path('/root/.cache/kagglehub/competitions/nfl-big-data-bowl-2026-analytics/114239_nfl_competition_files_published_analytics_final')

# Construct the full paths to the input, output and supplementary data
# NOTE: In a real scenario with all weeks, you'd loop through weeks 1-18
input_path = base_data_dir / 'train' / 'input_2023_w01.csv'
output_path = base_data_dir / 'train' / 'output_2023_w01.csv'
supp_path = base_data_dir / 'supplementary_data.csv'

# Load the data into pandas DataFrames
# Assuming the data is in CSV format based on earlier exploration
# NOTE: If the data were in parquet as suggested in the playbook's action,
# you would use pd.read_parquet() instead.
input_df = pd.read_csv(input_path)
output_df = pd.read_csv(output_path)
# Handle the DtypeWarning seen earlier by specifying dtype or using low_memory=False
supp_df = pd.read_csv(supp_path, low_memory=False)


# Normalize dtypes for join keys
for df in [input_df, output_df]:
    df['game_id'] = df['game_id'].astype('int64')
    df['play_id'] = df['play_id'].astype('int64')
    # Check if nfl_id exists before attempting to convert (ball has no nfl_id)
    if 'nfl_id' in df.columns:
        df['nfl_id'] = df['nfl_id'].astype('int64')

if 'game_id' in supp_df.columns and 'play_id' in supp_df.columns:
    supp_df['game_id'] = supp_df['game_id'].astype('int64')
    supp_df['play_id'] = supp_df['play_id'].astype('int64')


print("Shapes:", input_df.shape, output_df.shape, supp_df.shape)

Shapes: (285714, 23) (32088, 6) (18009, 41)


In [10]:
import pandas as pd, numpy as np, json, hashlib
from pathlib import Path
FRAME_RATE = 10
DT = 1.0 / FRAME_RATE
RANDOM_STATE = 100
OUTDIR = Path('data/interim')
OUTDIR.mkdir(parents=True, exist_ok=True)

In [12]:
input_df = input_df.rename(columns={'frame_id':'pre_frame_id'})
output_df = output_df.rename(columns={'frame_id':'post_frame_id'})

In [13]:
print("input dupes:", input_df.duplicated(['game_id','play_id','nfl_id','pre_frame_id']).sum())
print("output dupes:", output_df.duplicated(['game_id','play_id','nfl_id','post_frame_id']).sum())
print("supp dupes:", supp_df.duplicated(['game_id','play_id']).sum())

# Check basic value ranges for x,y
assert input_df['pre_frame_id'].min() >= 1
assert output_df['post_frame_id'].min() >= 1

input dupes: 0
output dupes: 0
supp dupes: 0


In [14]:
player_attrs = input_df[[
 'game_id','play_id','nfl_id','player_to_predict','player_name','player_height',
 'player_weight','player_birth_date','player_position','player_side','player_role',
 'num_frames_output','ball_land_x','ball_land_y'
]].drop_duplicates(subset=['game_id','play_id','nfl_id'])
# sanity
assert not player_attrs.duplicated(['game_id','play_id','nfl_id']).any()

In [16]:
reconstructed_rows = []
play_keys = sorted(set(input_df[['game_id','play_id']].itertuples(index=False, name=None)))

for game_id, play_id in play_keys:
    in_play = input_df[(input_df['game_id']==game_id)&(input_df['play_id']==play_id)].copy()
    out_play = output_df[(output_df['game_id']==game_id)&(output_df['play_id']==play_id)].copy()

    # if no output rows — skip or log (maybe play was scrimmaged out)
    if out_play.shape[0] == 0:
        # log and continue
        continue

    # per-player max pre_frame mapping
    last_pre = in_play.groupby('nfl_id')['pre_frame_id'].max().to_dict()

    # label phases
    in_play['phase'] = 'pre_throw'
    out_play['phase'] = 'post_throw'

    # for players that have no pre frames (rare), assume 0
    # create mapping for last_pre for each player in out_play
    out_play['last_pre'] = out_play['nfl_id'].map(last_pre).fillna(0).astype(int)
    out_play['frame_id'] = out_play['post_frame_id'] + out_play['last_pre']

    # merge static attributes into out_play (left)
    attrs_play = player_attrs[(player_attrs['game_id']==game_id)&(player_attrs['play_id']==play_id)]
    out_play = out_play.merge(attrs_play, on=['game_id','play_id','nfl_id'], how='left', validate='m:1')

    # drop any columns we don't want duplicated (post_frame_id kept for debug if desired)
    out_play = out_play.drop(columns=['last_pre'])

    # We purposely do NOT merge input frame x/y into out frames — keep input frames separate
    combined = pd.concat([in_play, out_play], ignore_index=True, sort=False)

    # sort by frame
    combined = combined.sort_values(['game_id','play_id','nfl_id', 'phase', 'pre_frame_id']).copy()
    reconstructed_rows.append(combined)

In [15]:
# build observed counts per player in output
obs_out = output_df.groupby(['game_id','play_id','nfl_id'])['post_frame_id'].max().reset_index().rename(columns={'post_frame_id':'obs_max_post'})
check = player_attrs.merge(obs_out, on=['game_id','play_id','nfl_id'], how='left')
check['match'] = check['obs_max_post'] == check['num_frames_output']
print("num_frames_output mismatch count:", (~check['match']).sum())
mismatches = check[~check['match']].sample(min(5, check[~check['match']].shape[0]), random_state=RANDOM_STATE)
print(mismatches[['game_id','play_id','nfl_id','num_frames_output','obs_max_post']])

num_frames_output mismatch count: 7410
         game_id  play_id  nfl_id  num_frames_output  obs_max_post
2987  2023091004      196   47816                 23           NaN
5379  2023091007     2160   49410                 18           NaN
2197  2023091002     3617   53481                  9           NaN
173   2023090700     1154   53487                 12           NaN
2485  2023091003     1782   55868                 16           NaN


In [18]:
# Step 7 — Combine all plays, apply final ordering & checks (Action)
merged_df = pd.concat(reconstructed_rows, ignore_index=True, sort=False)
# unify frame_id: input rows may have pre_frame_id; standardize a single frame_id column:
merged_df['frame_id'] = merged_df.apply(
    lambda r: r['pre_frame_id'] if pd.notna(r.get('pre_frame_id')) and r.get('phase')=='pre_throw' else r.get('frame_id'),
    axis=1
)

merged_df = merged_df.sort_values(['game_id','play_id','nfl_id','frame_id']).reset_index(drop=True)

In [19]:
# Step 7 — Combine all plays, apply final ordering & checks (Validation)

# 1) duplicates
assert not merged_df.duplicated(['game_id','play_id','nfl_id','frame_id']).any()

# 2) continuity count
gaps = merged_df.groupby(['game_id','play_id','nfl_id'])['frame_id'].apply(lambda x: (x.diff().dropna() != 1).sum()).sum()
print("total non-contiguous steps across players:", gaps)

total non-contiguous steps across players: 0


In [21]:
# if input has 'play_direction' column, merge it into merged_df per (game,play)
# Ensure we use the original input_df to get play_direction before any renames or merges might drop it
dir_map = input_df[['game_id','play_id','play_direction']].drop_duplicates().rename(columns={'play_direction':'original_play_direction'}) # Rename to avoid potential conflicts
merged_df = merged_df.merge(dir_map, on=['game_id','play_id'], how='left', validate='m:1')

# if play_direction == 'left', flip x: x_new = 120 - x_old (assuming 0-120)
# Use the original play direction column
mask = merged_df['original_play_direction'] == 'left'
merged_df.loc[mask, 'x'] = 120 - merged_df.loc[mask, 'x']
merged_df.loc[mask, 'ball_land_x'] = 120 - merged_df.loc[mask, 'ball_land_x']

# Optionally, rename the column back or drop the original_play_direction if not needed later
# merged_df = merged_df.rename(columns={'original_play_direction': 'play_direction'})

In [23]:
# Step 9 — Merge supplementary & use validate in merge (Validation)

# If validate raises, inspect duplicates in supp_df.
# Check final_df['pass_result'].isna().sum() — if many NaNs, some plays missing in supplementary.
print("frames missing pass_result:", final_df['pass_result'].isna().sum())

frames missing pass_result: 0


In [22]:
# Step 9 — Merge supplementary & use validate in merge (Action)
final_df = pd.merge(merged_df, supp_df, on=['game_id','play_id'], how='left', validate='m:1')

In [24]:
# Step 10 — Basic imputations / missing players handling (Action)

# Option A (conservative): keep these rows but mark player_name = 'UNKNOWN' and flag missing_attrs=True.
# Implementing Option A:
final_df['missing_attrs'] = final_df['player_name'].isna()
final_df['player_name'] = final_df['player_name'].fillna('UNKNOWN')

In [25]:
# Step 10 — Basic imputations / missing players handling (Validation)

# Validation: ensure missing attrs proportion is tiny; if large, investigate why.
# We can print the count of rows with missing attributes as the validation check.
print("rows with missing attrs after handling:", final_df['missing_attrs'].sum())

rows with missing attrs after handling: 0


In [26]:
# Step 12 — Quality Evaluation & Monitoring (Action is the checks themselves)

# 1) duplicates
assert not final_df.duplicated(['game_id','play_id','nfl_id','frame_id']).any()

# 2) continuity count
gap_counts = final_df.groupby(['game_id','play_id','nfl_id'])['frame_id'].apply(lambda x: (x.diff().dropna() != 1).sum())
print("players with gaps:", (gap_counts>0).sum())

# 3) expected frames check (compare num_frames_output)
expected_mismatch = final_df.groupby(['game_id','play_id','nfl_id']).agg(max_frame=('frame_id','max')).reset_index().merge(player_attrs[['game_id','play_id','nfl_id','num_frames_output']], on=['game_id','play_id','nfl_id'], how='left')
expected_mismatch['ok'] = (expected_mismatch['max_frame'] - expected_mismatch['num_frames_output'] == 0) | (expected_mismatch['num_frames_output'].isna())
print("expected mismatch count:", (~expected_mismatch['ok']).sum())

# 4) missing context
print("frames missing pass_result:", final_df['pass_result'].isna().sum())

players with gaps: 0
expected mismatch count: 10064
frames missing pass_result: 0


In [28]:
# Step 13 — Save safely with post-save verification & metadata (Validation)

# reload and compare row count and checksum:
df_check = pd.read_parquet(out_file)
assert len(df_check) == meta['rows']

# Optionally, recompute and compare checksum (more robust but slower)
# import hashlib
# with open(out_file, 'rb') as f:
#     data_check = f.read()
#     md5_check = hashlib.md5(data_check).hexdigest()
# assert md5_check == meta['md5']

print(f"Validation successful: Saved file '{out_file}' matches metadata.")

Validation successful: Saved file 'data/interim/reconstructed_week01.parquet' matches metadata.


In [27]:
# Step 13 — Save safely with post-save verification & metadata (Action)
out_file = OUTDIR / 'reconstructed_week01.parquet'
final_df.to_parquet(out_file, index=False, compression='snappy')

# compute md5 checksum
import hashlib
with open(out_file, 'rb') as f:
    data = f.read()
    md5 = hashlib.md5(data).hexdigest()

import json
meta = {
  'rows': len(final_df),
  'cols': list(final_df.columns),
  'md5': md5,
  'frame_rate': FRAME_RATE,
  'date': pd.Timestamp.now().isoformat(),
  'notes': 'merged input+output+supplementary; frame continuation; play_direction flipped to right-normalized; handled missing player attrs (Option A)'
}
with open(OUTDIR / 'reconstruct_week01_meta.json', 'w') as f:
    json.dump(meta, f, indent=2)

In [29]:
# Step 14 — Save small visual sample & logs for review (Action)

# Choose a few sample plays (replace with actual game_id and play_id from your data)
# Selecting plays that represent different scenarios (e.g., completed pass, incomplete pass, different pass lengths)
sample_play_keys = [
    (2023090700, 101),  # Example Play 1
    (2023091000, 1193), # Example Play 2
    (2023091002, 2693)  # Example Play 3
]

SAMPLE_OUTDIR = Path('data/samples')
SAMPLE_OUTDIR.mkdir(parents=True, exist_ok=True)

for game_id, play_id in sample_play_keys:
    sample_df = final_df[(final_df['game_id'] == game_id) & (final_df['play_id'] == play_id)].copy()
    sample_file = SAMPLE_OUTDIR / f'sample_play_{game_id}_{play_id}.parquet'
    sample_df.to_parquet(sample_file, index=False)
    print(f"Saved sample play {game_id}_{play_id} to {sample_file}")

# Create a simple merge log (can be expanded with more details)
log_file = OUTDIR / 'merge_log.txt'
with open(log_file, 'w') as f:
    f.write(f"Merge Log for reconstructed_week01.parquet\n")
    f.write(f"Date: {meta['date']}\n")
    f.write(f"Total rows in final_df: {meta['rows']}\n")
    f.write(f"Total players with gaps (Step 12): {gap_counts[gap_counts > 0].shape[0]}\n") # Use gap_counts from Step 12
    f.write(f"Total expected mismatch count (Step 12): {expected_mismatch[~expected_mismatch['ok']].shape[0]}\n") # Use expected_mismatch from Step 12
    f.write(f"Total frames missing pass_result (Step 12): {final_df['pass_result'].isna().sum()}\n")
    # Add more relevant counts or information here as needed
print(f"Saved merge log to {log_file}")

Saved sample play 2023090700_101 to data/samples/sample_play_2023090700_101.parquet
Saved sample play 2023091000_1193 to data/samples/sample_play_2023091000_1193.parquet
Saved sample play 2023091002_2693 to data/samples/sample_play_2023091002_2693.parquet
Saved merge log to data/interim/merge_log.txt


In [30]:
display(final_df.head())

,game_id,play_id,player_to_predict,nfl_id,pre_frame_id,play_direction_x,absolute_yardline_number,player_name,player_height,player_weight,...,penalty_yards,pre_penalty_yards_gained,yards_gained,expected_points,expected_points_added,pre_snap_home_team_win_probability,pre_snap_visitor_team_win_probability,home_team_win_probability_added,visitor_team_win_probility_added,missing_attrs
0,2023090700,101,False,43290,1.0,right,42.0,Jared Goff,6-4,223,...,NaN,0,0,0.927021,-2.145443,0.590426,0.409574,0.04972,-0.04972,False
1,2023090700,101,False,43290,2.0,right,42.0,Jared Goff,6-4,223,...,NaN,0,0,0.927021,-2.145443,0.590426,0.409574,0.04972,-0.04972,False
2,2023090700,101,False,43290,3.0,right,42.0,Jared Goff,6-4,223,...,NaN,0,0,0.927021,-2.145443,0.590426,0.409574,0.04972,-0.04972,False
3,2023090700,101,False,43290,4.0,right,42.0,Jared Goff,6-4,223,...,NaN,0,0,0.927021,-2.145443,0.590426,0.409574,0.04972,-0.04972,False
4,2023090700,101,False,43290,5.0,right,42.0,Jared Goff,6-4,223,...,NaN,0,0,0.927021,-2.145443,0.590426,0.409574,0.04972,-0.04972,False


In [32]:
# List all columns and their data types

# Set option to display all rows (all columns in this case)
pd.set_option('display.max_rows', None)

display(final_df.dtypes)

# Reset option to default
pd.reset_option('display.max_rows')

,0
game_id,int64
play_id,int64
player_to_predict,bool
nfl_id,int64
pre_frame_id,float64
play_direction_x,object
absolute_yardline_number,float64
player_name,object
player_height,object
player_weight,int64




Below I’ll first **list the important things we missed** (from the older guide) so you can see why they matter, then present the **updated playbook** that *includes them* (purpose → action → validation for every step). Use this as the canonical reference to run your notebooks.

---

## What the older guide had that we accidentally left out (and why each is important)

1. **Explicit dtype normalization** (game_id/play_id/nfl_id as int) — prevents silent merge mismatches.
2. **Use of `validate` parameter on merges** (e.g., `validate='m:1'`) — immediately alerts unexpected cardinality.
3. **num_frames_output vs actual output frames check** — ensures the input-provided expected lengths match reality.
4. **Handling missing players (present in output but not input)** — must decide backfill/flag logic.
5. **Simple reproducible sampling logic for test/visualization** (useful for small-parquet sample creation).
6. **Saving merge logs and metadata (rows, time, param values)** — essential for reproducibility and debugging.
7. **Basic error handling when saving files** (try/except + clear messaging).
8. **Small suggested diagnostic prints / counters** (how many plays merged, how many plays dropped) — for monitoring.
9. **Frame-rate assumption commented explicitly** and used consistently (e.g., 10Hz → dt=0.1s).
10. **A final quick save-check (load back and compare row counts/checksum)** to ensure file integrity after write.

All of those are now added into the final playbook below.

---

# 🔐 FINAL Merge & Reconstruction Playbook — Guided, Leak-Proof, Production Ready

> Copy-paste into a notebook and run step-by-step. Each step contains: Purpose → Action → Validation.

---

### GLOBAL NOTES (before you start)

* Default frame rate: `FRAME_RATE = 10` → `DT = 1 / FRAME_RATE = 0.1` seconds. Keep this constant.
* File layout (we expect):

  * `data/raw/week1_input.parquet`
  * `data/raw/week1_output.parquet`
  * `data/raw/supplementary.csv`
* Use small sample test runs first (e.g., 3 plays) before running full-week merges.

---

## Step 0 — Setup & constants

**Purpose:** reproducible environment & constants.

**Action:**

```python
import pandas as pd, numpy as np, json, hashlib
from pathlib import Path
FRAME_RATE = 10
DT = 1.0 / FRAME_RATE
RANDOM_STATE = 100
OUTDIR = Path('data/interim')
OUTDIR.mkdir(parents=True, exist_ok=True)
```

**Validation:** variables defined; OUTDIR exists.

---

## Step 1 — Load, type-normalize, quick shapes

**Purpose:** prevent silent mismatches due to dtype differences.

**Action:**

```python
input_df = pd.read_parquet('data/raw/week1_input.parquet')
output_df = pd.read_parquet('data/raw/week1_output.parquet')
supp_df = pd.read_csv('data/raw/supplementary_data.csv')

# Normalize dtypes for join keys
for df in [input_df, output_df]:
    df['game_id'] = df['game_id'].astype('int64')
    df['play_id'] = df['play_id'].astype('int64')
    df['nfl_id'] = df['nfl_id'].astype('int64')

supp_df['game_id'] = supp_df['game_id'].astype('int64')
supp_df['play_id'] = supp_df['play_id'].astype('int64')

print("Shapes:", input_df.shape, output_df.shape, supp_df.shape)
```

**Validation:**

* Print shows expected row counts.
* `input_df.dtypes[['game_id','play_id','nfl_id']]` are `int64`.

---

## Step 2 — Rename frame columns to avoid shadowing

**Purpose:** keep pre/post context explicit.

**Action:**

```python
input_df = input_df.rename(columns={'frame_id':'pre_frame_id'})
output_df = output_df.rename(columns={'frame_id':'post_frame_id'})
```

**Validation:** `pre_frame_id` present in input, `post_frame_id` in output.

---

## Step 3 — Uniqueness & basic sanity checks

**Purpose:** detect duplicates, obviously broken records before heavy work.

**Action:**

```python
print("input dupes:", input_df.duplicated(['game_id','play_id','nfl_id','pre_frame_id']).sum())
print("output dupes:", output_df.duplicated(['game_id','play_id','nfl_id','post_frame_id']).sum())
print("supp dupes:", supp_df.duplicated(['game_id','play_id']).sum())

# Check basic value ranges for x,y
assert input_df['pre_frame_id'].min() >= 1
assert output_df['post_frame_id'].min() >= 1
```

**Validation:** zero dupes ideally; if >0, inspect sample rows and log.

---

## Step 4 — Extract static player attributes (single per player-per-play)

**Purpose:** one source of truth for attributes, broadcast later.

**Action:**

```python
player_attrs = input_df[[
 'game_id','play_id','nfl_id','player_to_predict','player_name','player_height',
 'player_weight','player_birth_date','player_position','player_side','player_role',
 'num_frames_output','ball_land_x','ball_land_y'
]].drop_duplicates(subset=['game_id','play_id','nfl_id'])
# sanity
assert not player_attrs.duplicated(['game_id','play_id','nfl_id']).any()
```

**Validation:** `player_attrs` unique per (game,play,nfl).

---

## Step 5 — Validate `num_frames_output` expectation

**Purpose:** ensure the input's `num_frames_output` matches actual output frames (early sanity).

**Action:**

```python
# build observed counts per player in output
obs_out = output_df.groupby(['game_id','play_id','nfl_id'])['post_frame_id'].max().reset_index().rename(columns={'post_frame_id':'obs_max_post'})
check = player_attrs.merge(obs_out, on=['game_id','play_id','nfl_id'], how='left')
check['match'] = check['obs_max_post'] == check['num_frames_output']
print("num_frames_output mismatch count:", (~check['match']).sum())
mismatches = check[~check['match']].sample(min(5, check[~check['match']].shape[0]), random_state=RANDOM_STATE)
print(mismatches[['game_id','play_id','nfl_id','num_frames_output','obs_max_post']])
```

**Validation:** small mismatch count is tolerable; large mismatch → investigate (missing frames or input metadata issues). Log mismatches.

---

## Step 6 — Per-play loop: continue frames and align

**Purpose:** produce continuous timeline and keep throw boundary clear.

**Action (guided):**

```python
reconstructed_rows = []
play_keys = sorted(set(input_df[['game_id','play_id']].itertuples(index=False, name=None)))

for game_id, play_id in play_keys:
    in_play = input_df[(input_df['game_id']==game_id)&(input_df['play_id']==play_id)].copy()
    out_play = output_df[(output_df['game_id']==game_id)&(output_df['play_id']==play_id)].copy()

    # if no output rows — skip or log (maybe play was scrimmaged out)
    if out_play.shape[0] == 0:
        # log and continue
        continue

    # per-player max pre_frame mapping
    last_pre = in_play.groupby('nfl_id')['pre_frame_id'].max().to_dict()

    # label phases
    in_play['phase'] = 'pre_throw'
    out_play['phase'] = 'post_throw'

    # for players that have no pre frames (rare), assume 0
    # create mapping for last_pre for each player in out_play
    out_play['last_pre'] = out_play['nfl_id'].map(last_pre).fillna(0).astype(int)
    out_play['frame_id'] = out_play['post_frame_id'] + out_play['last_pre']

    # merge static attributes into out_play (left)
    attrs_play = player_attrs[(player_attrs['game_id']==game_id)&(player_attrs['play_id']==play_id)]
    out_play = out_play.merge(attrs_play, on=['game_id','play_id','nfl_id'], how='left', validate='m:1')

    # drop any columns we don't want duplicated (post_frame_id kept for debug if desired)
    out_play = out_play.drop(columns=['last_pre'])

    # We purposely do NOT merge input frame x/y into out frames — keep input frames separate
    combined = pd.concat([in_play, out_play], ignore_index=True, sort=False)

    # sort by frame
    combined = combined.sort_values(['game_id','play_id','nfl_id', 'phase', 'pre_frame_id']).copy()
    reconstructed_rows.append(combined)
```

**Validation:**

* Count how many plays processed vs total plays; log skipped plays.
* Spot-check several plays visually (next step).
* Ensure `frame_id` values in `out_play` start > `max(pre_frame_id)` for players who had pre frames.

---

## Step 7 — Combine all plays, apply final ordering & checks

**Purpose:** produce the single merged dataframe for the week.

**Action:**

```python
merged_df = pd.concat(reconstructed_rows, ignore_index=True, sort=False)
# unify frame_id: input rows may have pre_frame_id; standardize a single frame_id column:
merged_df['frame_id'] = merged_df.apply(
    lambda r: r['pre_frame_id'] if pd.notna(r.get('pre_frame_id')) and r.get('phase')=='pre_throw' else r.get('frame_id'),
    axis=1
)

merged_df = merged_df.sort_values(['game_id','play_id','nfl_id','frame_id']).reset_index(drop=True)
```

**Validation:**

* assert no duplicate quadruples:

```python
assert not merged_df.duplicated(['game_id','play_id','nfl_id','frame_id']).any()
```

* Check frame gaps per player:

```python
gaps = merged_df.groupby(['game_id','play_id','nfl_id'])['frame_id'].apply(lambda x: (x.diff().dropna() != 1).sum()).sum()
print("total non-contiguous steps across players:", gaps)
```

Expect small (0) gaps; if >0 inspect examples.

---

## Step 8 — Standardize field orientation (play direction)

**Purpose:** ensure coordinate frames are consistent (flip x for plays going left).

**Action:**

```python
# if input has 'play_direction' column, merge it into merged_df per (game,play)
dir_map = input_df[['game_id','play_id','play_direction']].drop_duplicates()
merged_df = merged_df.merge(dir_map, on=['game_id','play_id'], how='left', validate='m:1')

# if play_direction == 'left', flip x: x_new = 120 - x_old (assuming 0-120)
mask = merged_df['play_direction'] == 'left'
merged_df.loc[mask, 'x'] = 120 - merged_df.loc[mask, 'x']
merged_df.loc[mask, 'ball_land_x'] = 120 - merged_df.loc[mask, 'ball_land_x']
```

**Validation:** sample a few plays that have left direction and visually confirm flipping.

---

## Step 9 — Merge supplementary & use `validate` in merge

**Purpose:** attach play-level context safely and detect join cardinality issues.

**Action:**

```python
final_df = pd.merge(merged_df, supp_df, on=['game_id','play_id'], how='left', validate='m:1')
```

**Validation:**

* If `validate` raises, inspect duplicates in `supp_df`.
* Check `final_df['pass_result'].isna().sum()` — if many NaNs, some plays missing in supplementary.

---

## Step 10 — Basic imputations / missing players handling

**Purpose:** handle cases where output contains players who weren’t in input attrs.

**Action & Policy (decide one):**

* Option A (conservative): keep these rows but mark `player_name = 'UNKNOWN'` and flag `missing_attrs=True`.
* Option B (aggressive): drop these rows if they are irrelevant (e.g., ball-equipped objects).
  Implement:

```python
final_df['missing_attrs'] = final_df['player_name'].isna()
print("rows with missing attrs:", final_df['missing_attrs'].sum())
# fill or flag
final_df['player_name'] = final_df['player_name'].fillna('UNKNOWN')
```

**Validation:** ensure missing attrs proportion is tiny; if large, investigate why.

---

## Step 11 — Derive & smooth reconstruction-ready features (optional here)

**Purpose:** reconstruct speed, accel, heading from x,y on post-throw frames (do smoothing before computing derivatives).

**Action (example):**

```python
from scipy.signal import savgol_filter

def add_motion_features(df):
    df = df.sort_values('frame_id')
    dx = df['x'].diff().fillna(0)
    dy = df['y'].diff().fillna(0)
    df['speed_raw'] = np.sqrt(dx**2 + dy**2) * FRAME_RATE
    if len(df) > 5:
        df['speed'] = savgol_filter(df['speed_raw'].values, 5, 2)
    else:
        df['speed'] = df['speed_raw']
    df['accel'] = df['speed'].diff().fillna(0) * FRAME_RATE
    df['dir'] = np.degrees(np.arctan2(dy, dx))
    return df

final_df = final_df.groupby(['game_id','play_id','nfl_id']).apply(add_motion_features).reset_index(drop=True)
```

**Validation:** plot speed trace for sample players; check max speeds fall in plausible range (~0–12 yd/s).

---

## Step 12 — Quality Evaluation & Monitoring

**Purpose:** ensure no leak/mixups after full processing.

**Action (checks to run):**

```python
# 1) duplicates
assert not final_df.duplicated(['game_id','play_id','nfl_id','frame_id']).any()

# 2) continuity count
gap_counts = final_df.groupby(['game_id','play_id','nfl_id'])['frame_id'].apply(lambda x: (x.diff().dropna() != 1).sum())
print("players with gaps:", (gap_counts>0).sum())

# 3) expected frames check (compare num_frames_output)
expected_mismatch = final_df.groupby(['game_id','play_id','nfl_id']).agg(max_frame=('frame_id','max')).reset_index().merge(player_attrs[['game_id','play_id','nfl_id','num_frames_output']], on=['game_id','play_id','nfl_id'], how='left')
expected_mismatch['ok'] = (expected_mismatch['max_frame'] - expected_mismatch['num_frames_output'] == 0) | (expected_mismatch['num_frames_output'].isna())
print("expected mismatch count:", (~expected_mismatch['ok']).sum())

# 4) missing context
print("frames missing pass_result:", final_df['pass_result'].isna().sum())
```

**Validation:** Check counts are small and acceptable; document problematic plays.

---

## Step 13 — Save safely with post-save verification & metadata

**Purpose:** persist data and capture run metadata for reproducibility.

**Action:**

```python
out_file = OUTDIR / 'reconstructed_week01.parquet'
final_df.to_parquet(out_file, index=False, compression='snappy')

# compute md5 checksum
with open(out_file, 'rb') as f:
    data = f.read()
    md5 = hashlib.md5(data).hexdigest()

meta = {
  'rows': len(final_df),
  'cols': list(final_df.columns),
  'md5': md5,
  'frame_rate': FRAME_RATE,
  'date': pd.Timestamp.now().isoformat(),
  'notes': 'merged input+output+supplementary; frame continuation; play_direction flipped to right-normalized'
}
with open(OUTDIR / 'reconstruct_week01_meta.json', 'w') as f:
    json.dump(meta, f, indent=2)
```

**Validation:** reload and compare row count and checksum:

```python
df_check = pd.read_parquet(out_file)
assert len(df_check) == meta['rows']
```

---

## Step 14 — Save small visual sample & logs for review

**Purpose:** quick review artifact for manual spot-checks & CI.

**Action:**

* Save 3–5 sample plays (choose a variety: deep pass, short pass, contested) to `data/samples/`.
* Save a plain-text `merge_log.txt` with counts, mismatch examples, top issues.

**Validation:** open sample files, watch animations/plots to confirm pre->post continuity.

---

## Final Checklist (Before moving to metrics)

* [ ] All join keys normalized to ints.
* [ ] No duplicate `(game_id, play_id, nfl_id, frame_id)` rows.
* [ ] Frame continuity per player checked (gaps=0 or logged).
* [ ] `num_frames_output` vs actual checked and documented.
* [ ] play_direction normalized/flipped (if needed).
* [ ] Missing player-attrs handled/flagged.
* [ ] Save completed with metadata + checksum.
* [ ] Sample plays saved for human visual inspection.


